In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import faulthandler
import itertools as it
import python_example as olim
import matplotlib.pyplot as plt
import meshio
import numpy as np
import pyvista as pv

In [3]:
faulthandler.enable()

In [ ]:
mesh = meshio.read('sphere3.vtk')

points = mesh.points.astype(np.float64)
tetra = mesh.cells['tetra'].astype(np.int64)

S = lambda x: 1 + np.dot(x, [0.2, -0.3, 0.1])

slowness = S(points)

solver = olim.EikonalTetraSolver(points, tetra, slowness)

Find the index of the point nearest to the origin

In [63]:
for i, vt_neib in enumerate(solver.mesh.vt_neib):
    x0 = points[i]

    T = np.array(list(vt_neib))
    Is = np.array([np.setdiff1d(tetra[t], i) for t in T])

    for t, I in zip(T, Is):
        X = points[I] - x0
        dots = np.sum(X[[0, 1, 2]]*X[[1, 2, 0]], axis=1)
        
        if np.any(dots < 0):
            print('i = %d, t = %d, I = %s' % (i, t, I))
            assert False

i = 0, t = 2499, I = [ 803 1607 1898]


AssertionError: 

In [78]:
i = 0
t = 2499
I = [803, 1607, 1898]

In [82]:
X = points[I] - points[i]
dots = np.sum(X[[0, 1, 2]]*X[[1, 2, 0]], axis=1)
dots

array([-0.00089528,  0.00947447,  0.01634254])

In [108]:
tetra[np.array(solver.mesh.tt_neib[s])[[0, 2, 3]]]

array([[ 152, 1538,  803, 1607],
       [ 803, 1538, 1897, 1898],
       [1538, 1607, 1089, 1898]])

In [111]:
tetra[tt_neib]

array([[ 152,  803,    0, 1607],
       [   0,  803,   26, 1898],
       [ 803, 1607, 1538, 1898],
       [1433, 1607,    0, 1898]])

In [109]:
tt_neib = solver.mesh.tt_neib[t]
s = tt_neib[np.where(~np.any(tetra[tt_neib] == i, axis=1))[0][0]]
solver.mesh.tt_neib[s]

[3535, 2499, 4302, 1356]

In [27]:
i0 = np.argmin(np.sum(points**2, axis=1))

solver.set_state(i0, olim.State.Trial)
solver.set_value(i0, 0.0)

solver.commit()

print(solver.heap)

Int64Vector[954]


In [28]:
solver.step()

True

In [29]:
np.where(~np.isinf(solver.values))

(array([ 954, 1098, 1141, 1151, 1281, 1298, 1322, 1357, 1397, 1406, 1554,
        1645, 1690, 1729, 1810, 1855, 1904, 1908, 1989, 2001]),)

In [72]:
i, r = 200, 0

t = solver.tetra_nb[i][r]
T = tetra[t]
J = T[T != i]
x0, x1, x2 = points[J] - points[i]

x0@x1, x0@x2, x1@x2

(0.0007319406132258845, -0.0009253590231018986, 0.013193458878806073)

In [73]:
tetra[solver.mesh.tt_neib[t]] == 

array([[ 152,  803,    0, 1607],
       [   0,  803,   26, 1898],
       [ 803, 1607, 1538, 1898],
       [1433, 1607,    0, 1898]])

In [76]:
points[tetra[0]]

array([[ 0.40846406, -0.5025264 , -0.10550597],
       [ 0.50567876, -0.64659692, -0.13419885],
       [ 0.26854994, -0.66670308, -0.11423071],
       [ 0.37052469, -0.71675778, -0.21452937]])

In [16]:
grid = pv.read('sphere3.vtk')

In [23]:
centroids = np.mean(points[tetra], axis=1)

In [24]:
I = np.logical_and(
    np.any(np.array(solver.mesh.tt_neib) == -1, axis=1),
    centroids[:, 2] < 0
)

In [25]:
subgrid = grid.extract_cells(I)

In [26]:
plotter = pv.BackgroundPlotter()
plotter.add_mesh(subgrid, 'lightgrey', opacity=1, lighting=True, show_edges=True)
plotter.show()